# Tutorial 00: Computing features

In this notebook, we can see how to compute features making use of the Pipeline and Feature classes. First, we import a function to load data from multiple cities and the module that contains the code for features.

In [1]:
%load_ext autoreload
%autoreload 2
from damage.data import load_data_multiple_cities
from damage import features

Now, we start by reading the data for any cities we are interested in. In this case, we will load data from Aleppo.

In [2]:
## Reading
cities = ['raqqa']

rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)

The resulting object is a dictionary with the filenames preceded by a keyword (populated_areas, annotation, raster) as keys and data as values (dataframes, tifs, arrays...). By adding that keyword as a prefix we will be able to detect on a later step what type of data each key contains. 

The mapping between the city name (Aleppo in this case) and the corresponding filenames is done at damage/data/data_sources.py Ideally, we should create a standard way of naming the files so that file is not required.

In [3]:
data.keys()

dict_keys(['populated_areas_populated_areas.shp', 'annotation_4_Damage_Sites_Raqqa_CDA.shp', 'raster_raqqa_2013_01_17_zoom_19.tif', 'raster_raqqa_2014_03_21_zoom_19.tif', 'raster_raqqa_2015_02_02_zoom_19.tif', 'raster_raqqa_2016_07_01_zoom_19.tif', 'no_analysis_areas_5_No_Analysis_Areas_Raqqa.shp'])

In [4]:
# check unique dates
shapefile_df = data['annotation_4_Damage_Sites_Raqqa_CDA.shp']
print(shapefile_df.SensDt.unique())
print(shapefile_df.SensDt_2.unique())
print(shapefile_df.SensDt_3.unique())

['2013-10-22' None]
['2014-02-12' None]
['2015-05-29']


Now, let's get into the code of the Pipeline.

## The feature pipeline

The ___Pipeline___ class takes a __list of tuples__ (name, function) for preprocessors and features and applies them with the transform method. __The feature functions need to return a dataframe with an identically structured index so the merge can be performed (e.g. city, patch_id).__

The ___transform___ method __takes a dictionary of data__ where each key represents a different data source (e.g. annotations) and each value a data object (e.g. pandas dataframe). The transform         method __iterates first over the preprocessor functions, overwriting the data object__. Then, it __iterates over the feature functions creating new keys__ in the data dictionary with the passed name. That way, features can use data generates by previously computed features.

Finally, the transform method merges the data generates by the feature functions making use of the __common index structure__.


In [5]:
from functools import reduce
import pandas as pd

from damage.features.base import Transformer


class Pipeline(Transformer):

    def __init__(self, features, preprocessors):
        self.features = features
        self.feature_names = [feature_name for feature_name, _ in self.features]
        self.preprocessors = preprocessors

    def transform(self, data):
        for preprocessor_name, preprocessor in self.preprocessors:
            data = preprocessor(data)

        for feature_name, feature in self.features:
            data[feature_name] = feature(data)

        feature_data = [data[name] for name in self.feature_names if name in data.keys()]
        feature_data = self._merge_feature_data(feature_data)
        return feature_data

    def _merge_feature_data(self, feature_data):
        return reduce(lambda l, r: pd.merge(l, r, left_index=True, right_index=True, how='outer'), feature_data)


## A toy example

Let's go first with a toy example. First we create our data.

In [6]:
annotations = pd.DataFrame({
    'destroyed': [0, 1],
    'patch_id': ['1', '2'],
})
annotations

,destroyed,patch_id
0,0,1
1,1,2


In [7]:
image = pd.DataFrame({
    'image': ['image_a', 'image_b'],
    'patch_id': ['1', '2'],
})
image

,image,patch_id
0,image_a,1
1,image_b,2


In [8]:
data = {
    'annotations': annotations,
    'image': image
}

Now, we create some simple functions

In [9]:
def preprocessor_preprocess_annotations(data):
    annotation_data = data['annotations']
    data['annotations'] = annotation_data.rename(columns={'destroyed': 'damage'})
    return data

def feature_create_damage_dummy(data):
    annotation_data = data['annotations'].set_index('patch_id')
    damage_dummy = pd.get_dummies(annotation_data['damage'], drop_first=True, prefix='destroyed')
    return damage_dummy

def feature_split_images(data):
    image_data = data['image'].set_index('patch_id')
    # Split
    return image_data

Now we apply these functions to the data dictionary we created and we well get a pandas dataframe with our features, indexed by the common index.

In [10]:
pipeline = Pipeline(
    preprocessors=[
        ('preprocess_annotations', preprocessor_preprocess_annotations)
    ],
    features=[
        ('damage_dummy', feature_create_damage_dummy),
        ('feature_split_images', feature_split_images)
    ]
)
feature_data = pipeline.transform(data)
feature_data.head()

,destroyed_1,image
patch_id,,
1,0,image_a
2,1,image_b


## A real example

Next, we run a real example, with the data from daraa and some feature classes that exist in the damage library.

In [14]:
## Reading
cities = ['raqqa']

rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
for key in ['raster_raqqa_2014_03_21_zoom_19.tif', 'raster_raqqa_2016_07_01_zoom_19.tif']:
    data.pop(key)

In [15]:
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

feature_data = pipeline.transform(data)

INFO:::AnnotationPreprocessor:::2019-10-29 19:29:18,140:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-10-29 19:29:18,356:::Applying RasterSplitter
100%|██████████| 367/367 [01:56<00:00,  3.30it/s]
INFO:::AnnotationMaker:::2019-10-29 19:33:29,340:::Applying AnnotationMaker
INFO:::RasterPairMaker:::2019-10-29 19:33:32,247:::Applying RasterPairMaker


In [16]:
print(len(feature_data))
print(feature_data['destroyed'].isnull().sum())
print(feature_data.index.get_level_values('date').unique())

44404
222
DatetimeIndex(['2013-01-17', '2015-02-02'], dtype='datetime64[ns]', name='date', freq=None)


And now we can save this data as a pickle file to retrieve it later.

In [17]:
feature_data.to_pickle('../logs/features/example.p')

## Computing features for every city

## Damascus

In [1]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_damascus.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['damascus']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))



INFO:::AnnotationPreprocessor:::2019-07-31 22:07:06,122:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-31 22:07:07,317:::Applying RasterSplitter
100%|██████████| 183/183 [00:59<00:00,  9.27it/s]
INFO:::AnnotationMaker:::2019-07-31 22:09:14,806:::Applying AnnotationMaker
INFO:::RasterPairMaker:::2019-07-31 22:09:15,689:::Applying RasterPairMaker


Features stored in ../logs/features/features_damascus.p


### Daraa

In [6]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_daraa.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['daraa']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))


INFO:::AnnotationPreprocessor:::2019-07-29 19:23:14,109:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-29 19:23:14,381:::Applying RasterSplitter


  0%|          | 0/207 [00:00<?, ?it/s]

  6%|▋         | 13/207 [00:00<00:01, 126.59it/s]

  8%|▊         | 17/207 [00:00<00:03, 50.66it/s] 

 10%|█         | 21/207 [00:00<00:08, 23.14it/s]

 12%|█▏        | 24/207 [00:01<00:12, 14.99it/s]

 13%|█▎        | 27/207 [00:01<00:16, 11.24it/s]

 14%|█▍        | 29/207 [00:01<00:17,  9.95it/s]

 15%|█▍        | 31/207 [00:01<00:18,  9.50it/s]

 16%|█▌        | 33/207 [00:02<00:18,  9.24it/s]

 17%|█▋        | 35/207 [00:02<00:18,  9.37it/s]

 18%|█▊        | 37/207 [00:02<00:17,  9.53it/s]

 19%|█▉        | 39/207 [00:02<00:17,  9.71it/s]

 20%|█▉        | 41/207 [00:02<00:16,  9.89it/s]

 21%|██        | 43/207 [00:03<00:16,  9.98it/s]

 22%|██▏       | 45/207 [00:03<00:16, 10.01it/s]

 23%|██▎       | 47/207 [00:03<00:15, 10.01it/s]

 24%|██▎       | 49/207 [00:03<00:17,  8.

Features stored in ../logs/features/features_daraa.p


### Homs

In [8]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_homs.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['homs']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))



INFO:::AnnotationPreprocessor:::2019-07-31 22:42:15,204:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-31 22:42:16,467:::Applying RasterSplitter
100%|██████████| 323/323 [01:12<00:00, 28.99it/s]
INFO:::AnnotationMaker:::2019-07-31 22:45:19,364:::Applying AnnotationMaker
INFO:::RasterPairMaker:::2019-07-31 22:45:20,632:::Applying RasterPairMaker


### Raqqa

In [3]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_raqqa.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['raqqa']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))




INFO:::AnnotationPreprocessor:::2019-07-31 22:51:13,527:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-31 22:51:13,773:::Applying RasterSplitter


                            SiteID      SensDt     SensorID     ConfID  \
0     Building (General / Default)  2013-10-22  Worldview-1  Very High   
1     Building (General / Default)  2013-10-22  Worldview-1  Very High   
2     Building (General / Default)  2013-10-22  Worldview-1  Very High   
3     Building (General / Default)  2013-10-22  Worldview-1     Medium   
4     Building (General / Default)  2013-10-22  Worldview-1  Very High   
5     Building (General / Default)  2013-10-22  Worldview-1  Very High   
6     Building (General / Default)  2013-10-22  Worldview-1  Very High   
7     Building (General / Default)  2013-10-22  Worldview-1  Very High   
8     Building (General / Default)  2013-10-22  Worldview-1  Very High   
9     Building (General / Default)  2013-10-22  Worldview-1     Medium   
10    Building (General / Default)  2013-10-22  Worldview-1     Medium   
11    Building (General / Default)  2013-10-22  Worldview-1     Medium   
12    Building (General / Default)  20

100%|██████████| 367/367 [00:01<00:00, 304.30it/s]


KeyError: 'city'

# Idlib

In [4]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_idlib.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['idlib']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))





INFO:::AnnotationPreprocessor:::2019-07-30 10:42:51,547:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-30 10:42:51,777:::Applying RasterSplitter
100%|██████████| 167/167 [00:18<00:00,  8.85it/s]
INFO:::AnnotationMaker:::2019-07-30 10:43:39,102:::Applying AnnotationMaker
INFO:::RasterPairMaker:::2019-07-30 10:43:39,705:::Applying RasterPairMaker


Features stored in ../logs/features/features_idlib.p


# Deir

In [5]:
from damage.data import load_data_multiple_cities, save_as_pickle_to_path
from damage import features

file_name = 'features_deir.p'

## Reading
STORING_PATH = '../logs/features'
cities = ['deir']
rasters_path = '../data/city_rasters/'
annotations_path = '../data/annotations/'
polygons_path = '../data/polygons/'

data = load_data_multiple_cities(cities, rasters_path, annotations_path, polygons_path)
### Processing
from datetime import timedelta
patch_size = 64
stride = patch_size
TIME_TO_ANNOTATION_THRESHOLD = timedelta(weeks=1)
pipeline = features.Pipeline(
    preprocessors=[
        ('AnnotationPreprocessor', features.AnnotationPreprocessor()),
    ],
    features=[
        ('RasterSplitter', features.RasterSplitter(patch_size=patch_size, stride=stride)),
        ('AnnotationMaker', features.AnnotationMaker(patch_size, TIME_TO_ANNOTATION_THRESHOLD)),
        ('RasterPairMaker', features.RasterPairMaker()),
    ],

)

features = pipeline.transform(data)
save_as_pickle_to_path(features, '{}/{}'.format(STORING_PATH, file_name))
save_as_pickle_to_path(features[['destroyed', 'latitude', 'longitude']],
                       '{}/target_{}'.format(STORING_PATH, file_name))
print('Features stored in {}/{}'.format(STORING_PATH, file_name))

INFO:::AnnotationPreprocessor:::2019-07-30 10:44:39,868:::Applying AnnotationPreprocessor
INFO:::RasterSplitter:::2019-07-30 10:44:40,673:::Applying RasterSplitter
100%|██████████| 187/187 [00:36<00:00,  6.11it/s]
INFO:::AnnotationMaker:::2019-07-30 10:46:03,457:::Applying AnnotationMaker
INFO:::RasterPairMaker:::2019-07-30 10:46:03,872:::Applying RasterPairMaker


Features stored in ../logs/features/features_deir.p
